In [83]:
import warnings
import os
from tqdm import tqdm
from typing import List
import pandas as pd
from unidecode import unidecode
import plotly.express as px
import matplotlib.pyplot as plt

import unicodedata
import math
import json
import numpy as np
os.chdir("/Users/vophuhan/Everything/University/Năm 4/Semester 2/Ứng dụng Phân tích dữ liệu thông minh/21KHDL-TikTok-Analytics")


# from langdetect import detect
# from langdetect import DetectorFactory

warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
# Hiển thị 6 chữ số thập phân
pd.set_option('display.float_format', '{:.6f}'.format)

# Đọc dữ liệu


In [47]:
file_path = "data/cleaned_videos_with_weekly_rank.csv"

# Define only string columns
string_cols = ["author.id", "id", "music.id"]

# Create dtype mapping for string columns
dtype_schema = {col: "object" for col in string_cols}

# Read CSV with specified dtypes only for string columns
video_info_df = pd.read_csv(
    file_path, low_memory=False, dtype=dtype_schema)
video_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70996 entries, 0 to 70995
Data columns (total 60 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   author.downloadSetting           70996 non-null  float64
 1   author.duetSetting               70996 non-null  float64
 2   author.id                        70996 non-null  object 
 3   author.nickname                  70996 non-null  object 
 4   author.openFavorite              70996 non-null  bool   
 5   author.signature                 70140 non-null  object 
 6   author.stitchSetting             70996 non-null  float64
 7   author.uniqueId                  70996 non-null  object 
 8   author.verified                  70996 non-null  bool   
 9   authorStats.diggCount            70996 non-null  float64
 10  authorStats.followerCount        70996 non-null  float64
 11  authorStats.followingCount       70996 non-null  float64
 12  authorStats.heart 

Convert time cols.


In [48]:
# Chuyển timestamp sang dạng datetime UTC
video_info_df["createTime"] = pd.to_datetime(
    video_info_df["createTime"], utc=True)
video_info_df["collectTime"] = pd.to_datetime(
    video_info_df["collectTime"], utc=True)

# Chuyển sang múi giờ Việt Nam (UTC+7)
video_info_df["createTime"] = video_info_df["createTime"].dt.tz_convert(
    "Asia/Ho_Chi_Minh")
video_info_df["collectTime"] = video_info_df["collectTime"].dt.tz_convert(
    "Asia/Ho_Chi_Minh")

In [49]:
renamed_cols = {
    'author.uniqueId': 'username',
    "statsV2.playCount": "views",  # Lượt xem video
    "statsV2.diggCount": "likes",  # Lượt thích video
    "statsV2.commentCount": "comments",  # Lượt bình luận video
    "statsV2.shareCount": "shares",  # Lượt chia sẻ video
    'statsV2.collectCount': "collects",
    "statsV2.playCount_rank": "views_rank",  # Lượt xem video
    "statsV2.diggCount_rank": "likes_rank",  # Lượt thích video
    "statsV2.commentCount_rank": "comments_rank",  # Lượt bình luận video
    "statsV2.shareCount_rank": "shares_rank",  # Lượt chia sẻ video
    'statsV2.collectCount_rank': "collects_rank",
    'authorStats.diggCount': "user.likes",
    'authorStats.followerCount': "user.followers",
    'authorStats.followingCount': "user.followings",
    # 'authorStats.friendCount': "user.friends",
    'authorStats.heartCount': "user.hearts",
    'authorStats.videoCount': "user.videos",
}


df = video_info_df.rename(columns=renamed_cols)
df.sample(1)

,author.downloadSetting,author.duetSetting,author.id,author.nickname,author.openFavorite,author.signature,author.stitchSetting,username,author.verified,user.likes,user.followers,user.followings,authorStats.heart,user.hearts,user.videos,createTime,desc,diversificationId,id,isAd,music.authorName,music.duration,music.id,music.isCopyrighted,music.original,music.title,collects,comments,likes,views,shares,textLanguage,textTranslatable,video.VQScore,video.bitrate,video.claInfo.enableAutoCaption,video.claInfo.hasOriginalAudio,video.codecType,video.duration,video.encodedType,video.height,video.ratio,video.videoQuality,video.volumeInfo.Loudness,video.volumeInfo.Peak,video.width,collectTime,hashtags,num_hashtags,year,week,engagement_rate,weekly_score,weekly_score_rank,views_rank,engagement_rate_rank,shares_rank,comments_rank,likes_rank,collects_rank
54102,0.000000,0.000000,6910409263827862529,Long ăn gì?,False,⭐️Đã ăn lắm còn hay luyên thuyên..\n🌸Contact f...,0.000000,longlongneee,False,7671.000000,206000.000000,43.000000,7900000.000000,7900000.000000,1089.000000,2024-09-20 18:09:02+07:00,📍xôi cô chung: ngõ 45 hào nam. ⭐️: 10-40 cành....,10085.000000,7416680685332663570,False,Mitsu Sound,121.000000,6849968592285337602,False,False,Very cute melody by marimba tone(39813),499.000000,23.000000,1522.000000,55700.000000,427.000000,vi,True,68.760000,1816132.000000,True,True,h264,91.000000,normal,1024.000000,540p,normal,-17.500000,0.668340,576.000000,2025-03-19 16:53:41+07:00,"['ancungtiktok', 'learnontiktok', 'reviewanngo...",7,2024,38,0.044363,2.367596,564.000000,618,281,319,303,599,387


Drop video duration = 0


In [ ]:
df = df[df['video.duration'] > 0]

In [ ]:
time_th = pd.to_datetime("2024-01-01", utc=True)
df = df[df['createTime'] > time_th]

In [53]:
stats_cols = ['views',
              'likes',
              'comments',
              'shares',
              'collects',
              'engagement_rate'
              ]

rank_cols = [col for col in df.columns if col.endswith("_rank")]

In [113]:
top_video_weekly_df = df.sort_values(
    ['year', 'week', 'weekly_score_rank'], ascending=True).groupby(['year', 'week']).head(50)

In [146]:
max_index = top_video_weekly_df[top_video_weekly_df['year'] != 2025]['views'].idxmax()
top_video_weekly_df.loc[max_index]

author.downloadSetting               0.000000
author.duetSetting                   0.000000
author.id                 7377647868423619585
author.nickname                 Lê Thanh Tuấn
author.openFavorite                     False
                                 ...         
comments_rank                               1
likes_rank                                  1
collects_rank                               1
video_category                    Long (>30s)
video_length_category                 >5 mins
Name: 49291, Length: 62, dtype: object

In [ ]:
# Check necessary columns
filt_df = top_video_weekly_df
required_columns = ['video.duration', 'views', 'likes', 'comments', 'shares']
missing_columns = [
    col for col in required_columns if col not in filt_df.columns]
if missing_columns:
    raise ValueError(f"Missing columns: {missing_columns}")

In [ ]:
# Ensure bins are unique and correctly categorized
max_duration = filt_df['video.duration'].max()

bins = [0, 10, 30, 60, 120, 180, 240, 300, max_duration + 1]
labels = ['<10s', '10-30s', '30-60s', '1-2 mins',
          '2-3 mins', '3-4 mins', '4-5 mins', '>5 mins']

filt_df['video_length_category'] = pd.cut(
    filt_df['video.duration'], bins=bins, labels=labels, right=False)

In [139]:
# Insight 1.1: Views by Video Length Category
views_by_length = filt_df.groupby('video_length_category')[
    'views'].mean().reset_index()
fig1 = px.bar(views_by_length, x='video_length_category', y='views',
              title="Average Views by Video Length Category",
              labels={'statsV2.playCount': 'Average Views',
                      'video_length_category': 'Video Length Category'},
              color='video_length_category')
fig1.show()